In [54]:
import numpy as np
import pandas as pd
import cv2
import random
from pycocotools.coco import COCO
from sklearn.model_selection import train_test_split
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.decomposition import PCA, SparsePCA
from tqdm import tqdm

dataDir='data'
dataType='val2017'
annFile='{}/annotations/instances_{}.json'.format(dataDir,dataType)

# Initialize the COCO api for instance annotations
cocoData = COCO(annFile)

loading annotations into memory...
Done (t=1.20s)
creating index...
index created!


In [169]:
def segmentTo2DArray(segmentation):
    polygon = []
    for partition in segmentation:
        for x, y in zip(partition[::2], partition[1::2]):
            polygon.append((x, y))
    return polygon


def maskPixels(polygon, img_dict, image_folder):
    img = cv2.imread('{}/{}'.format(image_folder, img_dict['file_name']))
    mask = np.zeros(img.shape, dtype=np.uint8)
    polygon = np.int32(polygon)
    cv2.fillPoly(mask, [polygon], (255, 255, 255))
    masked_image = cv2.bitwise_and(img, mask)
    masked_image = cv2.cvtColor(masked_image, cv2.COLOR_BGR2RGB)
    return masked_image


def getSegmentedMasks(filterClasses, image_folder):
    # Returns single object annotation with black background
    catIds = cocoData.getCatIds(catNms=filterClasses)
    imgIds = cocoData.getImgIds(catIds=catIds)

    if len(imgIds) > 500:
        imgIds = random.sample(imgIds, 500)
    imgs = cocoData.loadImgs(imgIds)
    
    mask_annIds = []
    masked_imgs = []
    for img_dict in tqdm(imgs):
        # Load annotations
        annIds = cocoData.getAnnIds(imgIds=img_dict['id'], catIds=catIds, iscrowd=0)
        anns = cocoData.loadAnns(annIds)
        mask_annIds.extend(annIds)
        # Create masked images
        for ann in anns:
            polyVerts = segmentTo2DArray(ann['segmentation'])
            masked_img = maskPixels(polyVerts, img_dict, image_folder)
            valid_pix = np.float32(masked_img.reshape(-1, 3))
            valid_pix = valid_pix[np.all(valid_pix != 0, axis=1), :]
            if valid_pix.shape[0] > 0:
                masked_imgs.append(masked_img)
    return masked_imgs, mask_annIds

cat = 'traffic light'
print("Loading object masks...")
segmented_masks, mask_annIds = getSegmentedMasks([cat], 'data/val2017')

  1%|▊                                                                                 | 2/191 [00:00<00:14, 13.02it/s]

Loading object masks...


100%|████████████████████████████████████████████████████████████████████████████████| 191/191 [00:09<00:00, 20.82it/s]


In [170]:
def imageHist(image, bins=(4, 6, 3)):
    # compute a 3D color histogram over the image and normalize it
    hist = cv2.calcHist(image, [0, 1, 2], None, bins, [0, 180, 0, 256, 0, 256])
    hist = cv2.normalize(hist, hist).flatten()
    return hist

def loadHistograms(images, bins):
    data = []
    for image in tqdm(images):
        img_float32 = np.float32(image)
        image = cv2.cvtColor(img_float32, cv2.COLOR_RGB2HSV)
        features = imageHist(image, bins)
        data.append(features)
    return np.array(data)

print("Calculating histogram features...")
histData = loadHistograms(segmented_masks, bins=(3, 3, 3))

  3%|██▊                                                                             | 22/634 [00:00<00:02, 216.27it/s]

Calculating histogram features...


100%|███████████████████████████████████████████████████████████████████████████████| 634/634 [00:02<00:00, 251.76it/s]


In [171]:
def getArea(filterClasses, annIds=None):
    # Returns average proportion an object of a given class takes up in the image
    catIds = cocoData.getCatIds(catNms=filterClasses)
    imgIds = cocoData.getImgIds(catIds=catIds)

    data = {}
    for imgId in imgIds:
        imAnn = cocoData.loadImgs(ids=imgId)[0]
        all_annIds = cocoData.getAnnIds(imgIds=imgId, catIds=catIds, iscrowd=0)
        if annIds is not None:
            all_annIds = list(set(all_annIds).intersection(set(annIds)))
        objs = cocoData.loadAnns(ids=all_annIds)        
        for obj in objs:
            proportion = obj['area'] / (imAnn['width'] * imAnn['height'])
            data[len(data)] = [imgId, obj['id'], proportion]
    df = pd.DataFrame.from_dict(data, orient='index', columns=['imgID', 'annID', 'proportion of img'])
    avg = df['proportion of img'].sum() / len(df['proportion of img'])
    return avg, df


def getSegRoughness(filterClasses, annIds=None):
    # Returns average roughness an object of a given class
    # "Roughness" = number of segmentation vertices / area of obj
    catIds = cocoData.getCatIds(catNms=filterClasses)
    imgIds = cocoData.getImgIds(catIds=catIds)

    data = {}
    for imgID in imgIds:
        all_annIds = cocoData.getAnnIds(imgIds=imgID, catIds=catIds, iscrowd=0)
        if annIds is not None:
            all_annIds = list(set(all_annIds).intersection(set(annIds)))
        objs = cocoData.loadAnns(ids=all_annIds)
        for obj in objs:
            num_vertices = len(obj['segmentation'])
            roughness = num_vertices / obj['area']
            data[len(data)] = [imgID, obj['id'], roughness]
    df = pd.DataFrame.from_dict(data, orient='index', columns=['imgID', 'annID', 'roughness of annotation'])
    avg = df['roughness of annotation'].sum() / len(df['roughness of annotation'])
    return avg, df

print(len(mask_annIds))
print("Getting average area...")
avgArea, areaData = getArea([cat], mask_annIds)
print("Getting average roughness of segmentation...")
avgRoughness, roughnessData = getSegRoughness([cat], mask_annIds)

634
Getting average area...
Getting average roughness of segmentation...


In [175]:
def stichImages(im_list, interpolation=cv2.INTER_CUBIC):
    w_min = min(im.shape[1] for im in im_list)
    im_list_resize = [cv2.resize(im, (w_min, int(im.shape[0] * w_min / im.shape[1])), interpolation=interpolation)
                      for im in im_list]
    return np.array(cv2.vconcat(im_list_resize))

def getObjColors(image):
    n_colors = 4
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 20, 0.1)
    flags = cv2.KMEANS_RANDOM_CENTERS

    pixels = np.float32(image.reshape(-1, 3))
    pixels = pixels[np.all(pixels != 0, axis=1), :]
    if len(pixels) < n_colors:
        n_colors = len(pixels)
        print(n_colors)
    _, labels, palette = cv2.kmeans(pixels, n_colors, None, criteria, 10, flags)
    _, counts = np.unique(labels, return_counts=True)
    return counts, palette

def getCatColors(segmented_masks):
    print("--Processing object colours")
    colourData = []
    for mask in tqdm(segmented_masks):
        _, palette = getObjColors(mask)
        colourData.append(palette)
    print("--Stitching objects...")
    image = stichImages(segmented_masks)
    print("--Processing category colours...")
    counts, palette = getObjColors(image)
    # color_patch = displayDominantColors(counts, palette)
    return palette, colourData

print("Getting dominant colours...")
catColours, colourData = getCatColors(segmented_masks)

  1%|▊                                                                                 | 6/634 [00:00<00:10, 58.98it/s]

Getting dominant colours...
--Processing object colours


100%|████████████████████████████████████████████████████████████████████████████████| 634/634 [00:07<00:00, 80.23it/s]


--Stitching objects...
--Processing category colours...


In [139]:
def getOutliers(histData, areaData, roughnessData, colourData, nn=20, contamination=0.05,):
    colourData_2d = []
    for palette in colourData:
        c = [j for i in palette for j in i]
        colourData_2d.append(c)
    pca = PCA(n_components=3)
    colourData_2d = pca.fit_transform(colourData_2d)
    
    sparse_pca = SparsePCA(n_components=3)
    histData = sparse_pca.fit_transform(histData)
    
    train = pd.DataFrame(areaData['annID'])
    train = train.join(pd.DataFrame(histData, columns=['histX', 'histY', 'histZ']))
    train['area'] = areaData['proportion of img']
    train['roughness'] = roughnessData['roughness of annotation']
    train = train.join(pd.DataFrame(colourData_2d, columns=['colourX', 'colourY', 'colourZ']))
    train = train.drop(['annID'], axis=1)
    print(train.head)

    print("--Fitting anomaly detection model...")
    lof = LocalOutlierFactor(n_neighbors=nn, contamination=contamination)
    results = pd.DataFrame()
    results['lof'] = lof.fit_predict(train)
    results['negative_outlier_factor'] = lof.negative_outlier_factor_
    print(results.head)
    return results


def getAnomalies(filterClasses, preds):
    catIds = cocoData.getCatIds(catNms=filterClasses)
    imgIds = cocoData.getImgIds(catIds=catIds)
    annIds = cocoData.getAnnIds(imgIds=imgIds, catIds=catIds, iscrowd=0)

    outlying_objs_anns = []
    for annId, pred in zip(annIds, preds):
        if pred == -1:
            outlying_objs_anns.append(annId)

    imgs_with_outliers = []
    for img in imgIds:
        img_anns = set(cocoData.getAnnIds(imgIds=[img]))
        outlying_anns = set(outlying_objs_anns)
        if len(img_anns.intersection(outlying_anns)) > 0:
            imgs_with_outliers.append(img)
    return imgs_with_outliers, outlying_objs_anns


print("Getting abnormal objects...")
preds_df = getOutliers(histData, areaData, roughnessData, colourData)
outlier_imgIds, outlier_annIds = getAnomalies([cat], preds_df['lof'])
print(outlier_imgIds)
print(outlier_annIds)

Getting abnormal objects...
<bound method NDFrame.head of         histX  histY  histZ      area  roughness     colourX     colourY  \
0   -0.048586    0.0    0.0  0.013345   0.000548 -141.167080  -91.081141   
1   -0.048586    0.0    0.0  0.089747   0.000034   31.193865 -238.007377   
2   -0.048586    0.0    0.0  0.000744   0.004916   66.754082   85.825268   
3   -0.048586    0.0    0.0  0.001611   0.003311  193.500240 -154.081734   
4   -0.048586    0.0    0.0  0.020063   0.000532   -9.788154  219.335160   
..        ...    ...    ...       ...        ...         ...         ...   
419 -0.048586    0.0    0.0  0.020355   0.000294   48.338975  148.940904   
420 -0.048586    0.0    0.0  0.018989   0.000171   18.841282  -58.039602   
421 -0.048586    0.0    0.0  0.007071   0.000460    6.127668   23.549757   
422 -0.048586    0.0    0.0  0.002556   0.001273   72.816208   74.716849   
423 -0.048586    0.0    0.0  0.042531   0.000063 -109.508150  234.909089   

        colourZ  
0    -26.38

C:\Users\mrric\Anaconda3\lib\site-packages\sklearn\decomposition\sparse_pca.py:170: DeprecationWarning: normalize_components=False is a backward-compatible setting that implements a non-standard definition of sparse PCA. This compatibility mode will be removed in 0.22.
  DeprecationWarning)
